## 1. DATA LOADING & CLEANING

In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
data = pd.read_excel("gym recommendation.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: '/content/gc (1) (6).xlsx'

In [ ]:
data.columns

In [ ]:
data.drop(columns=['id'], inplace = True)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
print(data.dtypes)


In [ ]:
# Convert 'age', 'Height', 'weight', 'bmi' to numeric (case-sensitive names)
data[['age', 'Height', 'weight', 'bmi']] = data[['age', 'Height', 'weight', 'bmi']].apply(pd.to_numeric, errors='coerce')

# Handle missing values if any invalid values are coerced to NaN
# Option 1: Replace NaN with 0
data.fillna(0, inplace=True)

# Option 2: Drop rows with missing values in these columns
# data.dropna(subset=['age', 'Height', 'weight', 'bmi'], inplace=True)

# Verify the conversion and data types
print(data[['age', 'Height', 'weight', 'bmi']].dtypes)

# Optional: Check for any remaining NaN values
print(data[['age', 'Height', 'weight', 'bmi']].isnull().sum())


## 2. LABEL ENCODING

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_enc = LabelEncoder()

# Corrected column names based on your dataset
for col in ['sex', 'hypertension', 'diabetes', 'level', 'fitness_goal', 'fitness_type']:
    data[col] = label_enc.fit_transform(data[col])

# Verify the transformation
print(data[['sex', 'hypertension', 'diabetes', 'level', 'fitness_goal', 'fitness_type']].head())


In [ ]:
data.head()

## 3. NORMALIZATION

In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Normalize the numerical features with the correct column names
data[['age', 'Height', 'weight', 'bmi']] = scaler.fit_transform(data[['age', 'Height', 'weight', 'bmi']])

# Verify the transformation
print(data[['age', 'Height', 'weight', 'bmi']].head())


In [ ]:
data.head()

In [ ]:
# Select only numerical columns
numerical_data = data.select_dtypes(include=[np.number])

# Compute correlation matrix
corr_matrix = numerical_data.corr()

# Plot heatmap
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Correlation Heatmap")
plt.show()


In [ ]:
# Plot histograms for numerical columns
data.select_dtypes(include=[np.number]).hist(bins=20, figsize=(15, 10))
plt.suptitle("Distribution of Numerical Features")
plt.show()


In [ ]:
# Plot box plots for numerical columns
plt.figure(figsize=(15, 10))
sns.boxplot(data=data.select_dtypes(include=[np.number]))
plt.xticks(rotation=45)
plt.title("Box Plot of Numerical Features")
plt.show()


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

# Print column names to verify
print("Columns in the dataset:", data.columns)

# Correct column names based on your dataset
columns_to_check = ['age', 'Height', 'weight', 'bmi']

# Function to detect and remove outliers using Z-score method
def remove_outliers_zscore(data, columns, threshold=3):
    # Apply z-score on the specified columns
    z_scores = np.abs(zscore(data[columns].apply(pd.to_numeric, errors='coerce')))

    # Filter out rows where any of the columns exceed the threshold
    data_no_outliers = data[(z_scores < threshold).all(axis=1)]
    return data_no_outliers

# 1. Remove outliers using Z-score method
data_no_outliers = remove_outliers_zscore(data, columns_to_check, threshold=3)

# Check the cleaned data
print("Data after removing outliers:\n", data_no_outliers.head())


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# List of columns to visualize
columns_to_plot = ['age', 'Height', 'weight', 'bmi']

# Set the figure size for better clarity
plt.figure(figsize=(10, 6))

# Plot the boxplot for each feature
sns.boxplot(data=data[columns_to_plot])

# Title for the plot
plt.title("Boxplot for Age, Height, Weight, and BMI")

# Display the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Assuming `data` is your dataset with the necessary columns

# Prepare features for clustering
features = ['age', 'Height', 'weight', 'bmi', 'hypertension', 'diabetes', 'level', 'fitness_goal', 'fitness_type']
X = data[features]

# Ensure all features are in the same case as the original dataset (e.g., 'height' vs 'Height')
# Initialize and fit the scaler
scaler = StandardScaler()

# Fit scaler on the training data (assuming you have a separate training dataset, or fit it directly on your data)
X_scaled = scaler.fit_transform(X)

# Perform KMeans clustering (you can adjust the number of clusters)
kmeans = KMeans(n_clusters=4, random_state=42)
data['cluster'] = kmeans.fit_predict(X_scaled)

# Reduce dimensions for visualization (PCA to 2D)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Plot the clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=data['cluster'], palette='viridis', s=100, edgecolor='black')

# Set plot labels and title
plt.title("Clustering of Users Based on Features", fontsize=16)
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title="Cluster", loc="upper right")
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Corrected pairplot with column names matching your dataset
sns.pairplot(data, vars=['age', 'weight', 'bmi', 'Height'], hue='fitness_goal', palette='viridis')

# Add a title to the pairplot
plt.suptitle("Pairplot of Selected Features", y=1.02)

# Show the plot
plt.show()


In [ ]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Assuming `data` is your dataset and `scaler` is already fitted with your data's numeric columns.
scaler = StandardScaler()
numeric_columns = ['age', 'Height', 'weight', 'bmi']
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])

# Train K-Means model
kmeans = KMeans(n_clusters=5, random_state=42)
data['Cluster'] = kmeans.fit_predict(data[['sex', 'age', 'Height', 'weight', 'hypertension', 'diabetes', 'bmi', 'level', 'fitness_goal', 'fitness_type']])

def get_recommendation_kmeans():
    print("Please enter your details for a personalized workout and diet recommendation.")
    user_input = {
        'sex': int(input("Enter Sex (Male: 1 / Female: 0): ")),
        'age': float(input("Enter Age: ")),
        'Height': float(input("Enter Height in meters (e.g., 1.75): ")),
        'weight': float(input("Enter Weight in kg: ")),
        'hypertension': int(input("Do you have Hypertension (Yes: 1 / No: 0): ")),
        'diabetes': int(input("Do you have Diabetes (Yes: 1 / No: 0): ")),
        'bmi': float(input("Enter BMI: ")),
        'level': int(input("Enter Level (Underweight: 3 / Normal: 0 / Overweight: 2 / Obese: 1): ")),
        'fitness_goal': int(input("Enter Fitness Goal (Weight Gain: 0 / Weight Loss: 1): ")),
        'fitness_type': int(input("Enter Fitness Type (Muscular Fitness: 1 / Cardio Fitness: 0): "))
    }

    # Normalize numerical features
    user_df = pd.DataFrame([user_input])
    user_df[numeric_columns] = scaler.transform(user_df[numeric_columns])

    # Predict cluster for the user
    user_cluster = kmeans.predict(user_df[['sex', 'age', 'Height', 'weight', 'hypertension', 'diabetes', 'bmi', 'level', 'fitness_goal', 'fitness_type']])[0]
    cluster_data = data[data['Cluster'] == user_cluster]

    # Get the most common recommendations in the cluster
    recommendation = cluster_data[['exercises', 'equipment', 'diet']].mode().iloc[0]

    print("\nRecommended Workout and Diet Plan (K-Means):")
    print("EXERCISES:", recommendation['exercises'])
    print("EQUIPMENT:", recommendation['equipment'])
    print("DIET:", recommendation['diet'])

    # Collect feedback for MRR
    feedback_matrix = []
    feedback = int(input("Was this recommendation relevant? (Yes: 1 / No: 0): "))
    feedback_matrix.append(feedback)

    # Calculate MRR
    relevant_indices = [i + 1 for i, feedback in enumerate(feedback_matrix) if feedback == 1]
    if relevant_indices:
        mrr = np.mean([1 / rank for rank in relevant_indices])
    else:
        mrr = 0.0

    print(f"\nMean Reciprocal Rank (MRR) for K-Means: {mrr:.2f}")

    return recommendation

# Get and evaluate recommendations for K-Means
recommendation_kmeans = get_recommendation_kmeans()


In [ ]:
# import pickle
# with open('scaler.pkl', 'wb') as f:
#     pickle.dump(scaler, f)
# with open('kmeans.pkl', 'wb') as f:
#     pickle.dump(kmeans, f)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import numpy as np

# Assuming `data` is already loaded and cleaned
# Define the input features and separate targets
X = data[['sex', 'age', 'Height', 'weight', 'hypertension', 'diabetes', 'bmi', 'level', 'fitness_goal', 'fitness_type']]
y_exercises = data['exercises']
y_diet = data['diet']
y_equipment = data['equipment']

# Split the dataset into training and testing sets for each target
X_train, X_test, y_train_exercises, y_test_exercises = train_test_split(X, y_exercises, test_size=0.2, random_state=42)
_, _, y_train_diet, y_test_diet = train_test_split(X, y_diet, test_size=0.2, random_state=42)
_, _, y_train_equipment, y_test_equipment = train_test_split(X, y_equipment, test_size=0.2, random_state=42)

# Train Decision Tree models for each target
exercise_tree = DecisionTreeClassifier(random_state=42, max_depth=5)
diet_tree = DecisionTreeClassifier(random_state=42, max_depth=5)
equipment_tree = DecisionTreeClassifier(random_state=42, max_depth=5)

exercise_tree.fit(X_train, y_train_exercises)
diet_tree.fit(X_train, y_train_diet)
equipment_tree.fit(X_train, y_train_equipment)

# Evaluate the models
print("\nExercise Prediction Report:")
y_pred_exercises = exercise_tree.predict(X_test)
print(classification_report(y_test_exercises, y_pred_exercises, zero_division=0))
print(f"Exercise Prediction Accuracy: {accuracy_score(y_test_exercises, y_pred_exercises):.2f}")

print("\nDiet Prediction Report:")
y_pred_diet = diet_tree.predict(X_test)
print(classification_report(y_test_diet, y_pred_diet, zero_division=0))
print(f"Diet Prediction Accuracy: {accuracy_score(y_test_diet, y_pred_diet):.2f}")

print("\nEquipment Prediction Report:")
y_pred_equipment = equipment_tree.predict(X_test)
print(classification_report(y_test_equipment, y_pred_equipment, zero_division=0))
print(f"Equipment Prediction Accuracy: {accuracy_score(y_test_equipment, y_pred_equipment):.2f}")

# Function to get recommendations using the trained Decision Trees
def get_recommendation_decision_tree():
    print("\nPlease enter your details for a personalized workout recommendation.")
    user_input = {
        'sex': int(input("Enter Sex (Male : 1/Female : 0): ")),
        'age': float(input("Enter Age: ")),
        'Height': float(input("Enter Height in meters (e.g., 1.75): ")),
        'weight': float(input("Enter Weight in kg: ")),
        'hypertension': int(input("Do you have Hypertension (Yes : 1/No : 0): ")),
        'diabetes': int(input("Do you have Diabetes (Yes : 1/No : 0): ")),
        'bmi': float(input("Enter BMI: ")),
        'level': int(input("Enter Level (Underweight : 3, Normal : 0, Overweight : 2, Obese : 1): ")),
        'fitness_goal': int(input("Enter Fitness Goal (Weight Gain : 0, Weight Loss : 1): ")),
        'fitness_type': int(input("Enter Fitness Type (Muscular Fitness : 1, Cardio Fitness : 0): "))
    }

    # Convert user input to DataFrame
    user_df = pd.DataFrame([user_input])

    # Predict using the Decision Tree models
    recommended_exercises = exercise_tree.predict(user_df)[0]
    recommended_diet = diet_tree.predict(user_df)[0]
    recommended_equipment = equipment_tree.predict(user_df)[0]

    print("\nRecommended Workout Plan (Decision Tree):")
    print("EXERCISES:", recommended_exercises)
    print("DIET:", recommended_diet)
    print("EQUIPMENT:", recommended_equipment)

    # Collect feedback for MRR
    feedback_matrix = []

    # Assume recommendations are ranked from 1 to 3 (Exercises, Diet, Equipment)
    recommendations = [(recommended_exercises, 'Exercises'), (recommended_diet, 'Diet'), (recommended_equipment, 'Equipment')]

    for idx, (recommendation, category) in enumerate(recommendations, start=1):
        feedback = int(input(f"Was the {category} recommendation relevant? (Yes: 1, No: 0): "))
        feedback_matrix.append((idx, feedback))  # Store the rank and feedback

    # Calculate MRR (Mean Reciprocal Rank)
    relevant_indices = [idx for idx, feedback in feedback_matrix if feedback == 1]

    if relevant_indices:
        # MRR formula: 1 / rank for each relevant recommendation
        mrr = np.mean([1 / rank for rank in relevant_indices])  # Calculate MRR
    else:
        mrr = 0.0  # If no relevant recommendations

    print(f"\nMean Reciprocal Rank (MRR) for the recommendations: {mrr:.2f}")

    return recommended_exercises, recommended_diet, recommended_equipment, mrr


recommended_exercises, recommended_diet, recommended_equipment, mrr = get_recommendation_decision_tree()



  RANDOM **FOREST**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

# Assuming `data` is your dataset (replace this with your actual data)
# Preprocessing: Handling missing values and scaling
data = data.dropna()  # Remove rows with missing values

# Define numerical and categorical columns
num_features = ['age', 'Height', 'weight', 'bmi']
cat_features = ['sex', 'hypertension', 'diabetes', 'level', 'fitness_goal', 'fitness_type']

# Label encoding for categorical features
label_encoder = LabelEncoder()
for col in cat_features:
    data[col] = label_encoder.fit_transform(data[col])
import pickle
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)
# Separate features and target
X = data[num_features + cat_features]
y = data['recommendation']  # The target column

# Normalize numerical features
scaler = StandardScaler()
X[num_features] = scaler.fit_transform(X[num_features])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Predict on the test set
y_pred = rf.predict(X_test)

# Calculate accuracy (to check the performance)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Define function to get recommendation from Random Forest model
def get_recommendation_rf():
    print("Please enter your details for a personalized workout and diet recommendation.")

    # Collect user input
    user_input = {
        'sex': int(input("Enter Sex (Male : 1/Female : 0): ")),
        'age': float(input("Enter Age: ")),
        'Height': float(input("Enter Height in meters (e.g., 1.75): ")),
        'weight': float(input("Enter Weight in kg: ")),
        'hypertension': int(input("Do you have Hypertension (Yes : 1/No : 0): ")),
        'diabetes': int(input("Do you have Diabetes (Yes : 1/No : 0): ")),
        'bmi': float(input("Enter BMI: ")),
        'level': int(input("Enter Level (Underweight : 3, Normal : 0, Overweight : 2, Obese : 1): ")),
        'fitness_goal': int(input("Enter Fitness Goal (Weight Gain : 0, Weight Loss : 1): ")),
        'fitness_type': int(input("Enter Fitness Type (Muscular Fitness : 1, Cardio Fitness : 0): "))
    }

    # Convert the user input into a DataFrame
    user_df = pd.DataFrame([user_input])

    # Handle unseen categories in label encoding by checking for unseen levels
    for col in cat_features:
        if col == 'level':  # Handle the 'level' feature separately if it is unseen
            if user_input[col] not in label_encoder.classes_:
                print(f"Warning: The input for {col} ({user_input[col]}) is unseen during training. Assigning default value (0).")
                user_df[col] = 0  # Assign a default value (mode or other strategy) if unseen
        else:
            # Label encode categorical features to match the training set
            user_df[col] = label_encoder.transform(user_df[col])

    # Normalize numerical features to match the training set
    user_df[num_features] = scaler.transform(user_df[num_features])

    # Ensure the same feature order as the training set
    user_df = user_df[num_features + cat_features]

    # Predict recommendation
    prediction = rf.predict(user_df)

    # Display recommendation
    print("\nRecommended Workout and Diet Plan (Random Forest):")
    recommendation = data[data['recommendation'] == prediction[0]].iloc[0]
    print(f"EXERCISES: {recommendation['exercises']}")
    print(f"EQUIPMENTS: {recommendation['equipment']}")
    print(f"DIET: {recommendation['diet']}")

    # Collect feedback for MRR and NMRR
    feedback_matrix = []
    feedback = int(input("Was this recommendation relevant? (Yes: 1, No: 0): "))
    feedback_matrix.append(feedback)

    # Calculate MRR
    mrr = 0.0
    if feedback == 1:  # If feedback is relevant
        mrr = 1  # Since we're only considering one recommendation, its rank is 1

    print(f"\nMean Reciprocal Rank (MRR) for Random Forest: {mrr:.2f}")

    # Calculate NMRR (Normalized Mean Reciprocal Rank)
    relevant_indices = [i + 1 for i, feedback in enumerate(feedback_matrix) if feedback == 1]
    if relevant_indices:
        nmrr = np.mean([1 / rank for rank in relevant_indices])  # Calculate NMRR
    else:
        nmrr = 0.0  # If no relevant recommendations

    print(f"\nNormalized Mean Reciprocal Rank (NMRR) for Random Forest: {nmrr:.2f}")

    return recommendation

# Get and evaluate recommendations for Random Forest
recommendation_rf = get_recommendation_rf()


In [ ]:
import pickle
with open('scaler_random.pkl', 'wb') as f:
    pickle.dump(scaler, f)
with open('random.pkl', 'wb') as f:
    pickle.dump(kmeans, f)

## 4. RECOMMENDATION, FEEDBACK & EVALUATION

`Cosine similarity` is a metric used to measure how similar two vectors are
  - Once the cosine similarity scores are calculated between the new user's profile and existing users, the system identifies the top similar users (those with the highest similarity scores).
  - The system then generates recommendations based on the most common exercises and diets of these similar users, thereby tailoring suggestions to the new user's profile.

In [ ]:
# import random
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics.pairwise import cosine_similarity

# # Assuming 'data' is your dataset
# # Make sure the dataset 'data' is loaded with the correct column names

# # Fit the scaler on the dataset
# scaler = StandardScaler()
# data[['age', 'Height', 'weight', 'bmi']] = scaler.fit_transform(data[['age', 'Height', 'weight', 'bmi']])

# def get_recommendation(top_n=3):
#     print("Please enter your details for a personalized workout and diet recommendation.")
#     user_input = {
#         'sex': int(input("Enter Sex (Male : 1/Female : 0): ")),
#         'age': float(input("Enter Age: ")),
#         'Height': float(input("Enter Height in meters (e.g., 1.75): ")),
#         'weight': float(input("Enter Weight in kg: ")),
#         'hypertension': int(input("Do you have Hypertension (Yes : 1/No : 0): ")),
#         'diabetes': int(input("Do you have Diabetes (Yes : 1/No : 0): ")),
#         'bmi': float(input("Enter BMI: ")),
#         'level': int(input("Enter Level (Underweight : 3, Normal : 0, Overweight : 2, Obese : 1): ")),
#         'fitness_goal': int(input("Enter Fitness Goal (Weight Gain : 0, Weight Loss : 1): ")),
#         'fitness_type': int(input("Enter Fitness Type (Muscular Fitness : 1, Cardio Fitness : 0): "))
#     }

#     # Normalize numerical features
#     num_features = ['age', 'Height', 'weight', 'bmi']
#     user_df = pd.DataFrame([user_input], columns=num_features)
#     user_df[num_features] = scaler.transform(user_df[num_features])  # Normalize using the same scaler
#     user_input.update(user_df.iloc[0].to_dict())
#     user_df = pd.DataFrame([user_input])

#     # Calculate similarity scores for exact user input
#     user_features = data[['sex', 'age', 'Height', 'weight', 'hypertension', 'diabetes', 'bmi', 'level', 'fitness_goal', 'fitness_type']]
#     similarity_scores = cosine_similarity(user_features, user_df).flatten()

#     # Retrieve top similar users and get the first recommendation
#     similar_user_indices = similarity_scores.argsort()[-top_n:][::-1]
#     similar_users = data.iloc[similar_user_indices]
#     recommendation_1 = similar_users[['exercises', 'equipment', 'diet']].mode().iloc[0]  # Most common recommendation among top similar users

#     # Simulate two additional recommendations by modifying input values slightly
#     simulated_recommendations = []

#     for _ in range(2):
#         modified_input = user_input.copy()

#         # Randomly adjust Age, Weight, and BMI with larger variations
#         modified_input['age'] += random.randint(-5, 5)  # Adjust age by a larger range
#         modified_input['weight'] += random.uniform(-5, 5)  # Adjust weight by a larger range
#         modified_input['bmi'] += random.uniform(-1, 1)  # Adjust BMI by a larger range

#         # Normalize modified input values
#         modified_user_df = pd.DataFrame([modified_input], columns=num_features)
#         modified_user_df[num_features] = scaler.transform(modified_user_df[num_features])
#         modified_input.update(modified_user_df.iloc[0].to_dict())

#         # Calculate similarity scores for modified input
#         modified_similarity_scores = cosine_similarity(user_features, pd.DataFrame([modified_input])).flatten()
#         modified_similar_user_indices = modified_similarity_scores.argsort()[-top_n:][::-1]
#         modified_similar_users = data.iloc[modified_similar_user_indices]
#         recommendation = modified_similar_users[['exercises', 'equipment', 'diet']].mode().iloc[0]  # Get most common recommendation

#         # Check if the recommendation is already in simulated recommendations
#         if not any(rec['exercises'] == recommendation['exercises'] and rec['diet'] == recommendation['diet'] and rec['equipment'] == recommendation['equipment'] for rec in simulated_recommendations):
#             simulated_recommendations.append(recommendation)

#     # Display all recommendations
#     print("\nRecommended Workout and Diet Plans based on your input:")
#     print("\nRecommendation 1 (Exact match):")
#     print("EXERCISES:", recommendation_1['exercises'])
#     print("EQUIPMENTS:", recommendation_1['equipment'])
#     print("DIET:", recommendation_1['diet'])

#     for idx, rec in enumerate(simulated_recommendations, start=2):
#         print(f"\nRecommendation {idx} (Slight variation):")
#         print("EXERCISES:", rec['exercises'])
#         print("EQUIPMENTS:", rec['equipment'])
#         print("DIET:", rec['diet'])

#     # Collect feedback for each recommendation
#     feedback_matrix = []
#     for i in range(len(simulated_recommendations) + 1):  # +1 for the first recommendation
#         feedback = int(input(f"Was recommendation {i+1} relevant? (Yes: 1, No: 0): "))
#         feedback_matrix.append(feedback)

#     # Calculate MRR
#     relevant_indices = [i + 1 for i, feedback in enumerate(feedback_matrix) if feedback == 1]
#     if relevant_indices:
#         mrr = np.mean([1 / rank for rank in relevant_indices])  # Calculate MRR
#     else:
#         mrr = 0.0  # If no relevant recommendations

#     print(f"\nMean Reciprocal Rank (MRR): {mrr:.2f}")

#     return [recommendation_1] + simulated_recommendations, mrr

# # Get and display recommendations
# recommendations, mrr = get_recommendation(top_n=3)
